In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Importing Necessary Libraries

In [0]:
import glob
import cv2
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

Loading dataset paths for fetching images from drive

In [0]:
files = glob.glob("/content/gdrive/My Drive/VR/Project - Face recognition/AVR_data/**/*.jpg")
paths = []
names = []
 
for file in files:
  #Splitting path name for getting labels
  label_str = file.split("/")[-2]
  paths.append(file)
  names.append(label_str)

Function for data preprocessing

In [0]:
def preprocess(image):
  #converting images to grayscale
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  #resizing image to 400x400
  standard_im = cv2.resize(gray, (400, 400))
  return standard_im

Label Encoding of English names into numbers

In [0]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
names= label_encoder.fit_transform(names) 

Loading images from paths and passing them into preprocessing function defined above

In [0]:
data = []
for path in paths:
  #Reading image through imread function
  img = cv2.imread(path)
  #Preprocessing image
  pre = preprocess(img)
  data.append(pre)

Function for cropping face region out of the image and using it for further processing for face recognition.

In [0]:
def clean(data):
  face = []
  rect = []

  for img in data:
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow: Haar classifier
    face_cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/VR/Project - Face recognition/haarcascade_frontalface_default.xml')
  
    #let's detect multiscale images(some images may be closer to camera than others)
    #result is a list of faces
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)
  
    #if no faces are detected then return original img
    if(len(faces) == 0):
      face.append(None)
      rect.append(None)
  
    else:
    #under the assumption that there will be only one face,
    #extract the face area
      (x, y, w, h) = faces[0] 
  
    #return only the face part of the image
      face.append(img[y:y+w, x:x+h]) 
      rect.append(faces[0])
  return face,rect

Function to eleminate those images where no face was found by cascade classifier

In [0]:
def only_not_none(face,label,rect):
  face_list = []
  labels = []
  rect_list = []
  for face,label,rect in zip(face,names,rect):
    if face is not None:
      #add face to list of faces
      face_list.append(face)
      #add label for this face
      labels.append(label)
      #add rectangle co-ordinates of the face
      rect_list.append(rect)
  return face_list, labels, rect_list

In [0]:
face_data, rect_data = clean(data)
face_data,label_data,rect_data = only_not_none(face_data,names,rect_data)

Splitting Dataset

In [0]:
X_train, X_test, y_train, y_test = train_test_split(face_data, np.array(label_data), test_size=0.2, random_state=0)

Creating LBP Face recognizer model, training model and predicting values of the face labels and finally determining the accuracy of the model.

In [0]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()


#converting labels into numpy array
# nplabels = np.asarray(train_label)
# npface_list = np.asarray(train_face)

#training LBP model with training data
face_recognizer.train(X_train, y_train)

# Defining predict function
def predict_fun(test_data):
  pred = []
  for face in test_data:  
    #predict the image using our face recognizer 
    label = face_recognizer.predict(face)

    #storing label into list
    pred.append(label)
  return pred

#using predict function to predict labels
y_pred = predict_fun(X_test)

# Output of predict function gives labels and confidence. Hence, separating labels from confidence
pred_label = []
for i in y_pred:
  pred_label.append(int(i[0]))

#storing predicted labels into separate array and converting into int values
# final_y = []
# for i in pred_label:
#   final_y.append(int(i))

#Printing final accuracy
print("accuracy score:{:.2f}".format(metrics.accuracy_score(y_test, pred_label)))

accuracy score:0.87


In [0]:
print(classification_report(y_test, pred_label))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         4
           2       1.00      0.75      0.86         4
           3       1.00      0.67      0.80         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         1
           7       1.00      0.50      0.67         2
           8       0.75      0.75      0.75         4
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         4
          11       1.00      0.33      0.50         3
          12       0.50      1.00      0.67         1
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         1
          15       0.00      0.00      0.00         1
          16       0.60      1.00      0.75         3
          17       1.00      1.00      1.00         3
          18       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
